In [2]:
import numpy as np 
lst = [1, 4, 5, 6, 3, 2]

# 一次性热编码
max_value = max(lst)
one_hot_encoded = np.eye(max_value)[np.array(lst)-1]  # 减1以匹配0索引数组

print(one_hot_encoded)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [6]:
import torch
import torch.nn as nn

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        # 初始化双向LSTM
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        # 因为是双向，所以最后的全连接层输入尺寸是hidden_size的两倍
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        
    def forward(self, x):
        # 设置初始的隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device) # 乘以2因为是双向
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        print(x.shape)
        # 前向传播LSTM
        out, _ = self.lstm(x, (h0, c0))
        print(_[0].shape)
        # 解码最后一个时间步的隐藏状态
        out = self.fc(out[:, -1, :])
        return out

# 假设的参数
input_size = 10  # 输入特征维度大小
hidden_size = 128  # 隐藏层特征维度大小
num_layers = 2  # LSTM层的数量
num_classes = 3  # 输出的类别数

# 创建模型实例
model = BiRNN(input_size, hidden_size, num_layers, num_classes)

# 假设输入数据，这里只是一个示例
# 假设有一个批量大小为5，序列长度为7的输入，每个节点的特征向量大小为10
x = torch.randn(5, 7, input_size)

# 前向传播
outputs = model(x)
print(outputs)


torch.Size([5, 7, 10])
torch.Size([4, 5, 128])
tensor([[-0.0259,  0.0098, -0.0230],
        [-0.0358,  0.0197, -0.0207],
        [-0.0359,  0.0245, -0.0201],
        [-0.0367,  0.0136, -0.0197],
        [-0.0369,  0.0122, -0.0131]], grad_fn=<AddmmBackward0>)


In [6]:
import torch
import torch.nn as nn

class ARNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(ARNN, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # 因为是双向

    def forward(self, x, adj_matrix):
        # x: 节点特征矩阵，形状为(batch_size, num_nodes, feature_size)
        # adj_matrix: 图的邻接矩阵，形状为(num_nodes, num_nodes)

        updated_features = []
        for i in range(x.size(1)):  # 遍历所有节点
            neighbors_indices = (adj_matrix[i] > 0).nonzero(as_tuple=False).view(-1)
            node_features = x[:, i, :]  # 当前节点特征
            neighbor_features = [node_features.unsqueeze(0)]  # 包括节点自身，增加一个维度以匹配

            for neighbor_index in neighbors_indices:
                neighbor_features.append(x[:, neighbor_index, :].unsqueeze(0))  # 添加邻居节点特征

            # 计算平均特征向量
            neighbor_features = torch.cat(neighbor_features, dim=0)
            avg_feature = torch.mean(neighbor_features, dim=0, keepdim=True)
            updated_features.append(avg_feature.squeeze(0))  # 移除多余的维度

        # 将更新后的特征向量堆叠为一个新的特征矩阵
        updated_features = torch.stack(updated_features, dim=1)

        # 经过RNN
        out, _ = self.rnn(updated_features)
        # 取最后的输出
        out = self.fc(out[:, -1, :])
        return out

# 模型参数示例
input_size = 10
hidden_size = 128
num_layers = 2
output_size = 3

# 创建模型实例
arnn_model = ARNN(input_size, hidden_size, num_layers, output_size)

# 示例输入
batch_size = 1
num_nodes = 5
feature_size = 10
x = torch.randn(batch_size, num_nodes, feature_size)

# 示例邻接矩阵（这里需要根据实际计算图结构进行调整）
adj_matrix = torch.tensor([
    [0, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [1, 0, 0, 1, 1],
    [0, 1, 1, 0, 0],
    [0, 0, 1, 0, 0]
], dtype=torch.float32)

print(x.dtype, adj_matrix.dtype)
print(x.shape, adj_matrix.shape)
# 前向传播
outputs = arnn_model(x, adj_matrix)
print(outputs)


torch.float32 torch.float32
torch.Size([1, 5, 10]) torch.Size([5, 5])
tensor([[-0.0245,  0.0518, -0.0273]], grad_fn=<AddmmBackward0>)
